In [ ]:
#import libraries
from textblob import TextBlob
from google.cloud import translate
import json

In [ ]:
#edit your project details
project_id="your_project_id"
model_id ='general/base'
model_path = 'projects/{}/locations/{}/models/{}'.format(project_id, 'global', model_id)
keyFilename = 'path_to_filekey.json'

client = translate.TranslationServiceClient.from_service_account_json(keyFilename)
parent = client.location_path(project_id, "global")

tweet_list = []

In [ ]:
#translate function
def translate_text(text):
    response = client.translate_text(
        parent=parent,
        contents=[text],
        mime_type="text/plain",  
        source_language_code="hi",
        target_language_code="en-US",
        model=model_path
    )
    return(response.translations[0].translated_text)

In [ ]:
#get raw tweet data
with open('tweets.json', errors='ignore') as filehandle:
    tweet_rawlist = json.load(filehandle)

In [7]:
#make tweet list
for obj in tweet_rawlist:
    t_data = obj['tweet']
    lang = t_data['lang']
    text = t_data['full_text']
    
    #translate if language is 'hindi'
    if(lang == 'hi'):
        text = translate_text(text)
    elif(lang == 'und'):
        continue
    
    sentiment = TextBlob(text).sentiment_assessments
    polarity = sentiment.polarity
    subjectivity = sentiment.subjectivity
    score = sentiment.assessments
    t_dict = {
        'tweet': text, 
        'polarity': polarity,
        'subjectivity': subjectivity,
        'score': score,
        'url': f"https://twitter.com/your_twitter_username/status/{t_data['id']}"
    }
    tweet_list.append(t_dict)

In [ ]:
#Return polarity of tweet
def takePolarity(tweet):
    return tweet['polarity']

In [ ]:
#sort list in ascending order of polarity
new_list = sorted(tweet_list, key=takePolarity)

#print first 5 tweets 
for tweet in new_list[:5]:
    print(tweet, '\n')